使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第五節 電気ヒーター床暖房）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
from s04_01_general_Q import get_Q_T_H_d_t_i

## 第五節　電気ヒーター床暖房 / Section 5  Electric floor heating

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．最大暖房出力 / Maximum heating output

$$\large
Q_{max,H,d,t} = q_{max,H} \times A_{f} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h) / Maximum heating output per hour at time $ t $ of date $ d $  
$q_{max,H}$：単位面積当たりの上面最大放熱能力(W/m<sup>2</sup>) / Maximum top surface heat dissipation capacity per unit area   
$A_{f}$：敷設面積(m<sup>2</sup>) / Laying area  

単位面積当たりの上面最大放熱能力$q_{max,H}$は 162W/m<sup>2</sup> とする。

The top surface maximum heat dissipation capacity $ q_{max, H} $ per unit area is set to 162 W/m<sup>2</sup>.

In [ ]:
def get_Q_max_H_d_t(A_f):
    """最大暖房出力
    
    :param A_f: 敷設面積
    :type A_f: float
    :return: 最大暖房出力
    :rtype: ndarray
    """
    q_max_H = 162.0
    return np.ones(24 * 365) * q_max_H * A_f * 3600 * 10 ** (-6)  # (1)

### 6．暖房エネルギー消費量 / Heating energy consumption

#### 6.1　消費電力量 / Power consumption

$$
\large
E_{E,H,d,t} = Q_{T,H,d,t} \times \frac{1}{r_{up}} \times 10^{3} \div 3600
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h) / Power consumption per hour at time $ t $ of date $ d $  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における処理暖房負荷(MJ/h) / Processing heating load at time $ t $ of date $ d $   
$r_{up}$：上面放熱率 / Top surface heat release rate  

In [ ]:
def calc_E_E_H_d_t(A_f, r_up, L_H_d_t):
    """消費電力量
    
    :param A_f: 敷設面積
    :type A_f: float
    :param r_up: 温水床暖房の上面放熱率
    :type r_up: float
    :param L_H_d_t: 暖冷房区画の１時間当たりの暖房負荷
    :type L_H_d_t: ndarray
    :return: 消費電力量
    :rtype: ndarray
    """
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(A_f)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t)

    # 消費電力量
    E_E_H_d_t = Q_T_H_d_t / r_up * 10 ** 3 / 3600  # (2)

    return E_E_H_d_t

#### 6.2　ガス消費量 / Kerosene consumption

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は0とする。

It is assumed that gas consumption $ E_{G, H, d, t} $ per hour at time $ t $ of date $ d $ is zero.

In [ ]:
def get_E_G_H_d_t():
    """ガス消費量
    
    :return: ガス消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)

#### 6.3　灯油消費量 / Kerosene consumption

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は0とする。

It is assumed that kerosene consumption $ E_{K, H, d, t} $ per hour at time $ t $ of date $ d $ is zero.

In [ ]:
def get_E_K_H_d_t():
    """灯油消費量
    
    :return: 灯油消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)

#### 6.4　その他の燃料による一次エネルギー消費量 / Primary energy consumption by other fuels

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は0とする。

Primary energy consumption $ E_{M, H, d, t} $ by other fuel per hour at time $ t $ of date $ d $ is 0.

In [ ]:
def get_E_M_H_d_t():
    """その他の燃料による一次エネルギー消費量
    
    :return: その他の燃料による一次エネルギー消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)

## 付録 A 機器の性能を表す仕様の決定方法 / Appendix A How to determine the specification that represents the performance of the device

### A.1　記号及び単位 / Symbol and unit

省略

### A.2　敷設面積 / Laying area

$$\large
A_{f} = A_{HCZ} \times r_{Af}
$$

$A_{f}$：敷設面積(m<sup>2</sup>) / Laying area  
$A_{HCZ}$：電気ヒーター床暖房により暖房される暖冷房区画の床面積(m<sup>2</sup>) / Electric heating floor area of heating and cooling compartments heated by floor heating   
$r_{Af}$：床暖房パネルの敷設率 / Floor heating panel laying rate

In [ ]:
# 敷設面積 A_f
def get_A_f(A_HCZ, r_Af):
    """敷設面積
    
    :param A_HCZ: 暖冷房区画の床面積 (m2)
    :type A_HCZ: float
    :param r_Af: 床暖房パネルの敷設率
    :type r_Af: float
    :return: 敷設面積
    :rtype: float
    """
    return A_HCZ * r_Af  # (1)

$$\large
r_{Af} =  \frac{A_{f}}{A_{HCZ}}
$$

$A_{f}$：床暖房パネルの敷設面積(m<sup>2</sup>) / Floor heating panel laying area  
$A_{HCZ}$：電気ヒーター床暖房により暖房される暖冷房区画の床面積(m<sup>2</sup>) / Electric heating floor area of heating and cooling compartments heated by floor heating  

In [ ]:
# 床暖房パネルの敷設率 r_Af
def get_r_Af(A_f, A_HCZ):
    """[summary]
    
    :param A_f: 敷設面積
    :type A_f: float
    :param A_HCZ: 暖冷房区画の床面積 (m2)
    :type A_HCZ: float
    :return: 床暖房パネルの敷設率
    :rtype: float
    """
    return A_f / A_HCZ  # (2)

### A.3　上面放熱率 / Top surface heat release rate

$$\large
r_{up} =  \frac{(1-H) \times (R_{si}+R_{U})+(R_{P}+R_{D}+R_{se})}{R_{si}+R_{U}+R_{P}+R_{D}+R_{se}}\\
\\
\large = 1-H \times \frac{(R_{si}+R_{U})}{R_{si}+R_{U}+R_{P}+R_{D}+R_{se}}\\
\\
\large = 1-H \times (R_{si}+R_{U}) \times U
$$

$r_{up}$：上面放熱率 / Top surface heat release rate  
$R_{si}$：床暖房パネルの床上側表面熱伝達抵抗(m<sup>2</sup>K/W) / Floor top surface heat transfer resistance of floor heating panel  
$R_{U}$：床暖房パネル内の発熱体から床仕上げ材上側表面までの熱抵抗(m<sup>2</sup>K/W) / Thermal resistance from the heating element in the floor heating panel to the upper surface of the floor finish  
$R_{P}$：床暖房パネル内の発熱体から床暖房パネル床下側表面までの熱抵抗(m<sup>2</sup>K/W) / Thermal resistance from the heating element in the floor heating panel to the floor under surface of the floor heating panel  
$R_{D}$：床暖房パネルの下端から床下側表面までの熱抵抗(m<sup>2</sup>K/W) / Thermal resistance from the lower edge of the floor heating panel to the underfloor surface  
$R_{se}$：床下側表面熱伝達抵抗(m<sup>2</sup>K/W) / Underfloor surface heat transfer resistance  
$H$：温度差係数 / Temperature difference coefficient   
$U$：当該床の部位の熱貫流率(W/m<sup>2</sup>K) / Heat transmission coefficient of the area of the floor 

In [ ]:
def get_r_up():
    r_up = 1.0 - H * (R_si + R_U) * R_U     #(3)
    return (r_up * 100) / 100

### A.4　複数の電気ヒーター床暖房が設置される場合の仕様の決定方法 / How to determine the specifications when multiple electric heater floor heating is installed

省略

ommision